### Prepare geometry files for time series analysis 
1. concatenate multiple segments
2. downsample to full resolution, and save to the isce2 format
### Reference: 
- UAVSAR file format: https://uavsar.jpl.nasa.gov/science/documents/stack-format.html
### Notes:
- The notebook inputs require .lkv, .llh and one .ann files. 
- This notebook has been modified by Talib Oliver (https://github.com/taliboliver) from an original notebook created by Yunjun Zhang (https://github.com/yunjunz). 

In [1]:
%matplotlib inline
import os
import glob
import numpy as np
from matplotlib import pyplot as plt
from skimage.transform import resize
from mintpy.utils import readfile, writefile
plt.rcParams.update({'font.size': 12})

proj_dir  = os.path.expanduser('/mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s2/geom_reference')
os.makedirs(proj_dir, exist_ok=True)

os.chdir(proj_dir)
print('Go to directory:', proj_dir)

# inputs
dload_dir  = os.path.expanduser('/mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s2/download')

# output dir
geom_dir  = os.path.join(proj_dir, 'geometry1x1')
os.makedirs(geom_dir, exist_ok=True)

Go to directory: /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s2/geom_reference


In [2]:
# # link: https://uavsar.jpl.nasa.gov/science/documents/stack-format.html
# ann_file = glob.glob(os.path.join(dload_dir, '*.ann'))[0]
# ann_dict = readfile.read_uavsar_ann(ann_file)
# num_seg = int(ann_dict['Number of Segments'])
# # size info
# length, width = 0, 0
# for i in range(1,num_seg+1):
#     length += int(ann_dict[f'slc_{i}_1x1 Rows'])
#     width = int(ann_dict[f'slc_{i}_1x1 Columns'])
# print(f'full size: {length} x {width}')
# # flight info
# site, line = os.path.splitext(os.path.basename(ann_file))[0].split('_')[:2]
# ver, bcorr = os.path.splitext(os.path.basename(ann_file))[0].split('_')[-2:]

full size: 66664 x 9363


In [2]:
# link: https://uavsar.jpl.nasa.gov/science/documents/stack-format.html
ann_file = glob.glob(os.path.join(dload_dir, '*.ann'))[0]
ann_dict = readfile.read_uavsar_ann(ann_file)
# size info
length, width = 0, 0
length += int(ann_dict[f'slc_2_1x1 Rows'])
width = int(ann_dict[f'slc_2_1x1 Columns'])
print(f'full size: {length} x {width}')
# flight info
site, line = os.path.splitext(os.path.basename(ann_file))[0].split('_')[:2]
ver, bcorr = os.path.splitext(os.path.basename(ann_file))[0].split('_')[-2:]

full size: 55432 x 9363


### 1. Concatenate the <code>*.lkv/llh</code> files

In [3]:
# # rerun = False
# rerun = True
# for fext in ['.lkv', '.llh']:
#     out_file = os.path.join(dload_dir, f'{site}_{line}_{ver}_{bcorr}_2x8{fext}')
#     if rerun or not os.path.isfile(out_file):
#         for seg in range(1, num_seg+1):
#             seg_file = os.path.join(dload_dir, f'{site}_{line}_{ver}_{bcorr}_s{seg}_2x8{fext}')
#             with open(out_file, 'ab') as f:
#                 print(f'read data from file {os.path.basename(seg_file)} and append to file {os.path.basename(out_file)}')
#                 data = np.fromfile(seg_file, dtype=np.float32)
#                 f.write(data)

# # output file names
# lkv_file = os.path.join(dload_dir, f'{site}_{line}_{ver}_{bcorr}_2x8.lkv')
# llh_file = os.path.join(dload_dir, f'{site}_{line}_{ver}_{bcorr}_2x8.llh')

read data from file swatch_17304_01_BC_s1_2x8.lkv and append to file swatch_17304_01_BC_2x8.lkv
read data from file swatch_17304_01_BC_s1_2x8.llh and append to file swatch_17304_01_BC_2x8.llh


In [3]:
# rerun = False
rerun = True
for fext in ['.lkv', '.llh']:
    out_file = os.path.join(dload_dir, f'{site}_{line}_{ver}_{bcorr}_2x8{fext}')
    if rerun or not os.path.isfile(out_file):
        seg_file = os.path.join(dload_dir, f'{site}_{line}_{ver}_{bcorr}_s2_2x8{fext}')
        with open(out_file, 'ab') as f:
            print(f'read data from file {os.path.basename(seg_file)} and append to file {os.path.basename(out_file)}')
            data = np.fromfile(seg_file, dtype=np.float32)
            f.write(data)

# output file names
lkv_file = os.path.join(dload_dir, f'{site}_{line}_{ver}_{bcorr}_2x8.lkv')
llh_file = os.path.join(dload_dir, f'{site}_{line}_{ver}_{bcorr}_2x8.llh')

read data from file swatch_17304_01_BC_s2_2x8.lkv and append to file swatch_17304_01_BC_2x8.lkv
read data from file swatch_17304_01_BC_s2_2x8.llh and append to file swatch_17304_01_BC_2x8.llh


### 2. Crop, resize and convert to <code>ISCE-2</code> format

In [4]:
x0, y0 = 0, 0
rlooks, alooks = 2, 8 ## Current geometry file resolution,  #ALH Chng
x_step = int(rlooks) ### Reading a 2*8 geometry file that needs to be downsampled #ALH Chng
y_step = int(alooks) ###  #ALH Chng
x_num = width // x_step
y_num = length // y_step

# crop index with respect to the multilooked geometry
x0 = x0 // rlooks
y0 = y0 // alooks
x_step = x_step // rlooks
y_step = y_step // alooks


geom_dict = {
    'x_start' : x0,
    'y_start' : y0,
    'x_step'  : x_step,
    'y_step'  : y_step,
    'x_num'   : x_num,
    'y_num'   : y_num,
}
print(f'crop index: {geom_dict}')

crop index: {'x_start': 0, 'y_start': 0, 'x_step': 1, 'y_step': 1, 'x_num': 4681, 'y_num': 6929}


In [5]:
# read and resize
# kwargs = dict(output_shape=(length, width), order=1, mode='edge', preserve_range=True)
lat   = readfile.read(llh_file, datasetName='latitude' )[0][y0::y_step, x0::x_step][:y_num, :x_num]
lon   = readfile.read(llh_file, datasetName='longitude')[0][y0::y_step, x0::x_step][:y_num, :x_num]
hgt   = readfile.read(llh_file, datasetName='height'   )[0][y0::y_step, x0::x_step][:y_num, :x_num]
lkv_e = readfile.read(lkv_file, datasetName='east'     )[0][y0::y_step, x0::x_step][:y_num, :x_num]
lkv_n = readfile.read(lkv_file, datasetName='north'    )[0][y0::y_step, x0::x_step][:y_num, :x_num]
lkv_u = readfile.read(lkv_file, datasetName='up'       )[0][y0::y_step, x0::x_step][:y_num, :x_num]

# Down sample data to full res size
lat = resize(lat, (length, width), anti_aliasing=True)
lon = resize(lon, (length, width), anti_aliasing=True)
hgt = resize(hgt, (length, width), anti_aliasing=True)
lkv_e = resize(lkv_e, (length, width), anti_aliasing=True)
lkv_n = resize(lkv_n, (length, width), anti_aliasing=True)
lkv_u = resize(lkv_u, (length, width), anti_aliasing=True)

# normalize the look vector
lkv_amp = np.sqrt(lkv_e**2 + lkv_n**2 + lkv_u**2)
lkv_e /= lkv_amp
lkv_n /= lkv_amp
lkv_u /= lkv_amp

# look vector to LOS incidence and azimuth angle
inc_angle = np.rad2deg(np.arccos(lkv_u * -1))
az_angle = np.rad2deg(np.arcsin(lkv_e / np.sin(np.deg2rad(inc_angle)) * -1))

print ('Reading and resizeing done, file size and type is: ', lat.shape, lat.dtype)

Reading and resizeing done, file size and type is:  (55432, 9363) float32


In [6]:
# # plot for visual inspection
# fig, axs = plt.subplots(nrows=1, ncols=8, figsize=[15, 4.5], sharey=True)
# data_list = [lat, lon, hgt, lkv_e, lkv_n, lkv_u, inc_angle, az_angle]
# title_list = ['lat', 'lon', 'hgt', 'east', 'north', 'up', 'inc_angle', 'az_angle']
# for ax, data, title in zip(axs, data_list, title_list):
#     im = ax.imshow(data, interpolation='nearest');  fig.colorbar(im, ax=ax);  ax.set_title(title)
# fig.tight_layout()
# plt.show()

In [6]:
# output
hgt_file = os.path.join(geom_dir, 'hgt.rdr')
lat_file = os.path.join(geom_dir, 'lat.rdr')
lon_file = os.path.join(geom_dir, 'lon.rdr')
los_file = os.path.join(geom_dir, 'los.rdr')

meta={'BANDS': '1',
      'DATA_TYPE': 'float32',
      'FILE_TYPE': 'hgt',
      'FILE_LENGTH': str(lat.shape[0]), 
      'LENGTH': str(lat.shape[0]) ,
      'WIDTH': str(lat.shape[1]),
      'length': str(lat.shape[0]),
      'width': str(lat.shape[1])}

writefile.write(hgt, out_file=hgt_file, metadata=meta)  
writefile.write_isce_xml(meta, hgt_file)
writefile.write(lat, out_file=lat_file, metadata=meta)  
writefile.write_isce_xml(meta, lat_file)
writefile.write(lon, out_file=lon_file, metadata=meta)
writefile.write_isce_xml(meta, lon_file)

meta['BANDS'] = '2'
meta['INTERLEAVE'] = 'BIL'

ds_dict = {'incidenceAngle': inc_angle, 'azimuthAngle': az_angle}
writefile.write(ds_dict, out_file=los_file, metadata=meta)  
writefile.write_isce_xml(meta, los_file)

write file: /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s2/geom_reference/geometry1x1/hgt.rdr
write file: /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s2/geom_reference/geometry1x1/hgt.rdr.rsc
write file: /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s2/geom_reference/geometry1x1/hgt.rdr.xml
write file: /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s2/geom_reference/geometry1x1/hgt.rdr.vrt
write file: /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s2/geom_reference/geometry1x1/lat.rdr
write file: /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s2/geom_reference/geometry1x1/lat.rdr.rsc
write file: /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s2/geom_reference/geometry1x1/lat.rdr.xml
write file: /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s2/geom_reference/geometry1x1/lat.rdr.vrt
write file: /mnt/Backups/gbrench/repos/rg_uavsar/data/swatch_17304_s2/geom_reference/geometry1x1/lon.rdr
write file: /mnt/Backups/gbrenc

### Final Notes:
- Outputs can be multilooked using looks.py from ISCE or any other algorithm of choice. 

In [8]:
# !bash ./run_multilook_geometry.sh 8 3 #desired azimuth and range looks 